In [6]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
annual_stock = pd.read_csv('data/annual_stock.csv', encoding= 'GBK')
annual_stock.head()
annual_stock.shape

(24262, 362)

In [7]:
# transate all the column names into English with Google Translate
column_names = annual_stock.columns.values.tolist()

from googletrans import Translator

def translate_to_english(chinese_text):
    translator = Translator(service_urls=['translate.google.com'])
    translation = translator.translate(chinese_text, src='zh-cn', dest='en')
    return translation.text


# Translate each column name to English
english_column_names = [translate_to_english(column) for column in tqdm(column_names)]

# Print the translated column names
print(english_column_names)

# Rename the columns in the DataFrame
annual_stock.columns = english_column_names



100%|██████████| 362/362 [03:46<00:00,  1.60it/s]

['Stock number', 'Year (end of the year)', 'Total fixed assets', 'Interest -free liability', 'Interest -free non -current liabilities', 'Interest liabilities', 'Interest debt', 'Net debt', 'Formal asset', 'Operating capital', 'Net operating capital', 'Invest in capital', 'retained earnings', 'gross profit', 'Net income of business activities', 'Value change net income', 'Net interest cost', 'EBIT', 'Profit of profit tax folding and amortized', 'Profitable after -tax profit', 'Non -recurring profit or loss', 'The net profit attributable to shareholders of listed companies after deducting non -recurring profit or loss', 'Corporate free cash flow', 'Equity free cash flow', 'Depreciation and amortization', 'Earns per share (diluted at the end of the period, yuan/share)', 'Basic earnings per share', 'Dilute the income per share', 'Net assets per share (yuan/share)', 'Total income per share (yuan/share)', 'Business income per share (yuan/share)', 'Business profit per share (yuan/share)', 'Be

In [9]:
annual_stock.to_csv('data/annual_stock.csv', index=False, encoding='utf-8')